In [149]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot  as plt
from data import LoadSales
import logging as log
from models import *

In [167]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot  as plt
from data import LoadSales
import logging as log
from models import *
############################################
# >> Process start
log.info('** Load Data...')
SALES_df = LoadSales('teradata_sales')
list_ref = np.unique(SALES_df.dataframe.NUM_ART)
RESULT_FORECAST = pd.DataFrame()
# data split
n_test = 14 
step = 12


In [177]:
for i in list_ref[0:13] :
    ref = list_ref[i]
    
    

IndexError: index 36633331 is out of bounds for axis 0 with size 292

In [180]:
for ref in list_ref[0:13] :
        print(ref)
    

36633331
60191215
60191530
60388524
60388552
60388573
60388580
60417196
60454576
60454590
61117735
62182680
62183023
